# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [52]:
# Read in the avocado data
import pandas as pd

# Reading tab-delimited data
avocado = pd.read_csv('data/avocado.csv', sep='\t')

# Subsetting large DataFrame
relevant_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']

avocado = avocado[relevant_columns]
avocado.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


In [53]:
# Reading data from TXT files for relevant category tags
path = 'data/relevant_avocado_categories.txt'
with open(path, 'r') as file:
    relevant_categories = file.read().splitlines()
    file.close()
    
relevant_categories

['en:avocadoes',
 'en:avocados',
 'en:fresh-foods',
 'en:fresh-vegetables',
 'en:fruchte',
 'en:fruits',
 'en:raw-green-avocados',
 'en:tropical-fruits',
 'en:tropische-fruchte',
 'en:vegetables-based-foods',
 'fr:hass-avocados']

In [54]:
# Filter avocado data using relevant category tags

# Dropping rows with null values in a particular column
avocado = avocado.dropna(subset=['categories_tags'])

# Turning a column of comma separated tags into a column of lists
avocado['categories_list'] = avocado['categories_tags'].str.split(',')

# Filtering a DataFrame based on a column of lists
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]

avocado.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags,categories_list
5,3662994002063,fr,NaN,3 fruits,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
6,8437013031011,fr,NaN,1 kg,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
14,4016249238155,de,NaN,135g,100g,de:gläschen,Allos,allos,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,Europäische Union,en:european-union,"[en:plant-based-foods-and-beverages, en:plant-..."
17,8718963381532,de,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Deutschland,en:germany,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
23,8436002746707,es,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,España,en:spain,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."


In [55]:
# Where do most UK avocados come from?

# Filtering your DataFrame by a particular country
avocado_uk = avocado[(avocado['countries'] == 'United Kingdom')]

# Counting and ordering by the unique values in the country of origin column
# Get the country with the highest count
top_avocado_origin = avocado_uk['origins_tags'].value_counts().index[0]

# Strip out characters before country name
top_avocado_origin = top_avocado_origin.lstrip("en:")
top_avocado_origin = top_avocado_origin.replace('-', ' ')

In [56]:
# Create a user-defined function to call for each ingredient

# Creating function to call for each ingredient
def read_and_filter_data(filename, relevant_categories):
    #  Read the file
    df = pd.read_csv('data/' + filename, sep='\t')
    
    # Subset to just the relevant columns
    subset_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
    
    df = df[subset_columns]
    
    # Split categories tags into lists
    path = 'data/' + relevant_categories
    with open(path, 'r') as file:
        relevant_categories = file.read().splitlines()
        file.close()
        
    # Drop rows with null categories values 
    df = df.dropna(subset=['categories_tags'])
    df['categories_list'] = df['categories_tags'].str.split(',')
    
    # Filter DataFrame based on column of lists
    df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
    
    # Filter for data where countries equals "United Kingdom."
    df_uk = df[df['countries'] == 'United Kingdom']
    
    # Count and order by the unique values in the country of origin column
    # Get the country with the highest count
    top_origin_string = df_uk['origins_tags'].value_counts().index[0]
    
    # Clean up the country string data
    top_origin_country = top_origin_string.lstrip('en:')
    top_origin_country = top_origin_country.replace('-', ' ')
    
    return print(top_origin_country)

In [57]:
# Read relevant categories data file and call function for each ingredient
top_olive_oil_origin = read_and_filter_data('olive_oil.csv', 'relevant_olive_oil_categories.txt')

top_sourdough_origin = read_and_filter_data('sourdough.csv', 'relevant_sourdough_categories.txt')

greece
united kingdom
